In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [2]:
from load_librairies import *
import xscale.signal.fitting as xsf


dir_hist_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/historical/tos/atl3/'
dir_hist_z23 = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/historical/z20/atl3/'

dir_rcp8_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp585/tos/atl3/'
dir_rcp8_z23 = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp585/z20/atl3/'

path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/'
lon_min = -20
lon_max = 0
lat_min = -3
lat_max = 3

period_str_1 = 1950
period_end_1 = 1999

period_str_2 = 2050
period_end_2 = 2099
now = datetime.now()
date_time = now.strftime("%d/%m/%Y")
def is_jja(month):
    return (month >= 6) & (month <= 8)

def nandetrend(y):
    ''' Remove the linear trend from the data '''
    x = np.arange(0,y.shape[0],1)
    
    not_nan_ind = ~np.isnan(y)
    m, b, r_val, p_val, std_err = stats.linregress(x[not_nan_ind],np.array(y)[not_nan_ind])
    
    
    y_detrended_tmp= np.array(y) - m*x - b
    
    y_detrended  = xr.Dataset({'detrended': (['time'],y_detrended_tmp)}
                       ,coords={'time':y.time[:]})
    
    return y_detrended


path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'

In [3]:
#model = ['ACCESS-CM2',
#         'ACCESS-ESM1-5', 'BCC-CSM2-MR',
#         'CAMS-CSM1-0', 'CanESM5', 'EC-Earth3', 'EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
#         'INM-CM4-8', 'INM-CM5-0',
#         'IPSL-CM6A-LR', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0','NESM3']

model =  ['ACCESS-CM2',  'ACCESS-ESM1-5', 'BCC-CSM2-MR',  'CAMS-CSM1-0', 'CanESM5', 'EC-Earth3',
  'EC-Earth3-Veg','GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0',  'IPSL-CM6A-LR', 'MIROC6',
  'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3']


# Historical

In [4]:


alpha_bf3_hist_jja = np.ones((len(model)))
r_bf3_hist_jja = np.ones((len(model)))
std_bf3_hist_jja = np.ones((len(model)))
p_value_bf3_hist_jja = np.ones((len(model)))


alpha_bf3_hist_monthly = np.ones((len(model),12))
r_bf3_hist_monthly = np.ones((len(model),12))
std_bf3_hist_monthly = np.ones((len(model),12))
p_value_bf3_hist_monthly = np.ones((len(model),12))



for i in range(0,len(model)):
    
    sst = xr.open_dataset(dir_hist_sst+'tos_Omon_'+model[i]+'_historical_r1i1p1f1_187001-201412_1deg_atl3.nc')
       
    try:

        sst = sst.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
    except TypeError:
        sst['time'] = sst.indexes['time'].to_datetimeindex()
        sst = sst.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
    sst = sst.tos.mean(dim='lon').mean(dim='lat').chunk(chunks=None)
    #sst = xsf.detrend(sst[:],dim='time',type='linear')
    sst = nandetrend(sst)
    sst = sst.detrended.load()
    ssta, _ = Atools.ano_norm_t(sst)
    ssta_atl3 = ssta

    z23 = xr.open_dataset(dir_hist_z23+'z20_Omon_'+model[i]+'_historical_r1i1p1f1_187001-201412_1deg_atl3.nc')
    
    print(model[i])

    try:

        z23 = z23.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
    except TypeError:
        z23['time'] = z23.indexes['time'].to_datetimeindex()
        z23 = z23.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
    z23_tmp = z23.z20.mean(dim='lon').mean(dim='lat').chunk(chunks=None)
    z23  = nandetrend(z23_tmp)
    z23a, _ = Atools.ano_norm_t(z23.detrended.load())
    
    z23a_atl3_tmp = z23a
    idx = np.isfinite(z23a_atl3_tmp)
    z23a_atl3 = z23a_atl3_tmp[idx]
    ssta_atl3 = ssta_atl3[np.array(idx)]

    ssta_atl3_jja = ssta_atl3.sel(time=is_jja(ssta_atl3['time.month']))
    z23a_atl3_jja = z23a_atl3.sel(time=is_jja(z23a_atl3['time.month']))

    alpha_bf3_hist_jja[i], _, r_bf3_hist_jja[i], p_value_bf3_hist_jja[i], std_bf3_hist_jja[i] = stats.linregress(
    z23a_atl3_jja[:],ssta_atl3_jja[:])
    
    
    for j in range(0,12):
        alpha_bf3_hist_monthly[i,j], _, r_bf3_hist_monthly[i,j], p_value_bf3_hist_monthly[i,j], std_bf3_hist_monthly[i,j] = stats.linregress(
    z23a_atl3.sel(time=z23a_atl3['time.month'] == j+1), ssta_atl3.sel(time=ssta_atl3['time.month'] == j+1))
     

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


ACCESS-CM2


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


ACCESS-ESM1-5


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-pac

...

MPI-ESM1-2-LR


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


MRI-ESM2-0


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-pac

NESM3


In [5]:
bf3_hist_jja  = xr.Dataset({'alpha': (['product'],alpha_bf3_hist_jja),
                       'r': (['product'],r_bf3_hist_jja),
                       'p_value': (['product'],p_value_bf3_hist_jja),
                       'std': (['product'],std_bf3_hist_jja)}
                       ,coords={'product':model},
                        attrs={'standard_name': 'BF3',
                                    'long_name': 'Third Bjerknes feedback component',
                                    'units': 'K/m',
                                    'model': 'CMIP6',
                                    'Scenario': 'Historical',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})



In [6]:

bf3_hist_jja.to_netcdf(path_data+'bf3_hist_jja_CMIP6_z20.nc',mode='w')

## RCP85

In [7]:

alpha_bf3_rcp8_jja = np.ones((len(model)))
r_bf3_rcp8_jja = np.ones((len(model)))
std_bf3_rcp8_jja = np.ones((len(model)))
p_value_bf3_rcp8_jja = np.ones((len(model)))


alpha_bf3_rcp8_monthly = np.ones((len(model),12))
r_bf3_rcp8_monthly = np.ones((len(model),12))
std_bf3_rcp8_monthly = np.ones((len(model),12))
p_value_bf3_rcp8_monthly = np.ones((len(model),12))


model_id = []

for i in range(0,len(model)):
    
    sst = xr.open_dataset(dir_rcp8_sst+'tos_Omon_'+model[i]+'_ssp585_r1i1p1f1_201501-209912_1deg_atl3.nc')

    try:

        sst = sst.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
    except TypeError:
        sst['time'] = sst.indexes['time'].to_datetimeindex()
        sst = sst.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31))) 

    sst = sst.tos.mean(dim='lon').mean(dim='lat').chunk(chunks=None)
    sst = nandetrend(sst)
    sst = sst.detrended.load()
    ssta, _ = Atools.ano_norm_t(sst)
    ssta_atl3 = ssta


    
    z23 = xr.open_dataset(dir_rcp8_z23+'z20_'+model[i]+'_ssp585_r1i1p1f1_201501-209912_1deg_atl3.nc')
   
    print(model[i])

    try:

        z23 = z23.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
    except TypeError:
        z23['time'] = z23.indexes['time'].to_datetimeindex()
        z23 = z23.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31))) 
    z23_tmp = z23.z20.mean(dim='lon').mean(dim='lat').chunk(chunks=None)
    z23  = nandetrend(z23_tmp)
    z23a, _ = Atools.ano_norm_t(z23.detrended.load())
    
    z23a_atl3_tmp = z23a
    idx = np.isfinite(z23a_atl3_tmp)
    z23a_atl3 = z23a_atl3_tmp[idx]
    ssta_atl3 = ssta_atl3[np.array(idx)]

    ssta_atl3_jja = ssta_atl3.sel(time=is_jja(ssta_atl3['time.month']))
    z23a_atl3_jja = z23a_atl3.sel(time=is_jja(z23a_atl3['time.month']))

    alpha_bf3_rcp8_jja[i], _, r_bf3_rcp8_jja[i], p_value_bf3_rcp8_jja[i], std_bf3_rcp8_jja[i] = stats.linregress(
     z23a_atl3_jja[:],ssta_atl3_jja[:])
    
    
    
    #for j in range(0,12):
    #    alpha_bf3_rcp8_monthly[i,j], _, r_bf3_rcp8_monthly[i,j], p_value_bf3_rcp8_monthly[i,j], std_bf3_rcp8_monthly[i,j] = stats.linregress(
    #z23a_atl3.sel(time=z23a_atl3['time.month'] == j+1), ssta_atl3.sel(time=ssta_atl3['time.month'] == j+1))
     

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


ACCESS-CM2


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the fu

ACCESS-ESM1-5
BCC-CSM2-MR


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-pac

...

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


MRI-ESM2-0


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


NESM3


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


In [9]:
bf3_rcp8_jja  = xr.Dataset({'alpha': (['product'],alpha_bf3_rcp8_jja),
                       'r': (['product'],r_bf3_rcp8_jja),
                       'p_value': (['product'],p_value_bf3_rcp8_jja),
                       'std': (['product'],std_bf3_rcp8_jja)}
                       ,coords={'product':model},
                        attrs={'standard_name': 'BF3',
                                    'long_name': 'Third Bjerknes feedback component',
                                    'units': 'K/m',
                                    'model': 'CMIP6',
                                    'Scenario': 'SSP5 8.5',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})




In [10]:
bf3_rcp8_jja.to_netcdf(path_data+'bf3_ssp5_jja_CMIP6_z20.nc',mode='w')